<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png"  width="300" alt="cognitiveclass.ai logo"  />
</center>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction

Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones.

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.

Therefore if we can determine if the first stage will land, we can determine the cost of a launch.

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

<a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01" target="_blank">Spacex DataSet</a>


### Store the dataset in database table

**it is highly recommended to manually load the table using the database console LOAD tool in DB2**.

<img src = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/spacexload.png">

Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the  dataset, Next create a New Table, and then follow the steps on-screen instructions to load the data. Name the new table as follows:

**SPACEXDATASET**

**Follow these steps while using old DB2 UI which is having Open Console Screen**

**Note:While loading Spacex dataset, ensure that detect datatypes is disabled. Later click on the pencil icon(edit option).**

1.  Change the Date Format by manually typing DD-MM-YYYY and timestamp format as DD-MM-YYYY HH\:MM:SS

2.  Change the PAYLOAD_MASS\_\_KG\_  datatype  to INTEGER.

<img src = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/spacexload2.png">


**Changes to be considered when having DB2 instance with the new UI having Go to UI screen**

*   Refer to this insruction in this <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sign%20up%20for%20IBM%20Cloud%20-%20Create%20Db2%20service%20instance%20-%20Get%20started%20with%20the%20Db2%20console/instructional-labs.md.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">link</a> for viewing  the new  Go to UI screen.

*   Later click on **Data link(below SQL)**  in the Go to UI screen  and click on **Load Data** tab.

*   Later browse for the downloaded spacex file.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/browsefile.png" width="800"/>

*   Once done select the schema andload the file.

 <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/spacexload3.png" width="800"/>


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [3]:
%load_ext sql

**DB2 magic in case of old UI service credentials.**

In the next cell enter your db2 connection string. Recall you created Service Credentials for your Db2 instance before. From the **uri** field of your Db2 service credentials copy everything after db2:// (except the double quote at the end) and paste it in the cell below after ibm_db_sa://

<img src ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_edX/images/URI.jpg">

in the following format

**%sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name**

**DB2 magic in case of new UI service credentials.**

<img src ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/servicecredentials.png" width=600>  

*   Use the following format.

*   Add security=SSL at the end

**%sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name?security=SSL**


In [12]:
import ibm_db
import ibm_db_dbi
import pandas as pd
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud" # e.g.: "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = "pql21806"        # e.g. "abc12345"
dsn_pwd = "ifLEN1k36u8s3Alj"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "31198"                # e.g. "32733" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"              #i.e. "SSL"


#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

pconn = ibm_db_dbi.Connection(conn)

Connected to database:  BLUDB as user:  pql21806 on host:  0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud


In [18]:
qr = 'select * from spacex'
pd.read_sql(qr, pconn).head()

,DATE,TIME__UTC_,BOOSTER_VERSION,LAUNCH_SITE,PAYLOAD,PAYLOAD_MASS__KG_,ORBIT,CUSTOMER,MISSION_OUTCOME,LANDING__OUTCOME
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


## Tasks

Now write and execute SQL queries to solve the assignment tasks.

### Task 1

##### Display the names of the unique launch sites  in the space mission


In [88]:
qr = 'select unique(LAUNCH_SITE) from spacex'
pd.read_sql(qr, pconn)

,LAUNCH_SITE
0,CCAFS LC-40
1,CCAFS SLC-40
2,CCAFSSLC-40
3,KSC LC-39A
4,VAFB SLC-4E


### Task 2

##### Display 5 records where launch sites begin with the string 'CCA'


In [16]:
qr = "select LAUNCH_SITE from spacex where LAUNCH_SITE like 'CCA%' limit 5"
pd.read_sql(qr, pconn)

,LAUNCH_SITE
0,CCAFS LC-40
1,CCAFS LC-40
2,CCAFS LC-40
3,CCAFS LC-40
4,CCAFS LC-40


### Task 3

##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [40]:
qr = """
SELECT  CUSTOMER, sum(PAYLOAD_MASS__KG_) as TOTAL_SUM  
FROM spacex 
WHERE CUSTOMER = 'NASA (CRS)' 
GROUP BY CUSTOMER
"""
pd.read_sql(qr, pconn)

,CUSTOMER,TOTAL_SUM
0,NASA (CRS),45596


### Task 4

##### Display average payload mass carried by booster version F9 v1.1


In [50]:
qr = """
SELECT  AVG(PAYLOAD_MASS__KG_) as average_payload_mass
FROM spacex 
WHERE BOOSTER_VERSION like 'F9 v1.1%' 

"""
pd.read_sql(qr, pconn)

,AVERAGE_PAYLOAD_MASS
0,2534


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.

*Hint:Use min function*


In [52]:
qr = """
SELECT  min(DATE) as First_landing
FROM spacex 
WHERE LANDING__OUTCOME like 'Success%' 

"""
pd.read_sql(qr, pconn)

,FIRST_LANDING
0,2015-12-22


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [55]:
qr = """
SELECT  BOOSTER_VERSION, PAYLOAD_MASS__KG_ ,LANDING__OUTCOME
FROM spacex 
WHERE (PAYLOAD_MASS__KG_ >4000 AND PAYLOAD_MASS__KG_ <6000)
AND LANDING__OUTCOME = 'Success (drone ship)'

"""
pd.read_sql(qr, pconn)

,BOOSTER_VERSION,PAYLOAD_MASS__KG_,LANDING__OUTCOME
0,F9 FT B1022,4696,Success (drone ship)
1,F9 FT B1026,4600,Success (drone ship)
2,F9 FT B1021.2,5300,Success (drone ship)
3,F9 FT B1031.2,5200,Success (drone ship)


### Task 7

##### List the total number of successful and failure mission outcomes


In [60]:
qr = """
SELECT s.success, f.failure FROM 
   (SELECT COUNT( MISSION_OUTCOME ) success FROM spacex WHERE MISSION_OUTCOME like 'Success%' ) s, 
   (SELECT COUNT( MISSION_OUTCOME ) failure FROM spacex WHERE MISSION_OUTCOME not like 'Success%' ) f

"""
pd.read_sql(qr, pconn)

,SUCCESS,FAILURE
0,100,1


### Task 8

##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [67]:
qr = """
SELECT  BOOSTER_VERSION
FROM spacex 
WHERE PAYLOAD_MASS__KG_ =
    ( select max(PAYLOAD_MASS__KG_)	FROM spacex )

"""
pd.read_sql(qr, pconn)

,BOOSTER_VERSION
0,F9 B5 B1048.4
1,F9 B5 B1049.4
2,F9 B5 B1051.3
3,F9 B5 B1056.4
4,F9 B5 B1048.5
5,F9 B5 B1051.4
6,F9 B5 B1049.5
7,F9 B5 B1060.2
8,F9 B5 B1058.3
9,F9 B5 B1051.6


### Task 9

##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015


In [74]:
qr = """
SELECT  MONTHNAME(DATE) as month_name, LANDING__OUTCOME, BOOSTER_VERSION, LAUNCH_SITE
FROM spacex 
WHERE year(DATE) = 2015 and LANDING__OUTCOME = 'Failure (drone ship)'

"""
pd.read_sql(qr, pconn)

,MONTH_NAME,LANDING__OUTCOME,BOOSTER_VERSION,LAUNCH_SITE
0,January,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
1,April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### Task 10

##### Rank the  count of  successful landing_outcomes between the date 2010-06-04 and 2017-03-20 in descending order.


In [86]:
qr = """
SELECT  LANDING__OUTCOME, COUNT(*) as Count 
FROM spacex 
WHERE DATE >= '2010-06-04' and DATE <= '2017-03-20' and LANDING__OUTCOME like 'Success%'
GROUP BY LANDING__OUTCOME
order by Count
"""
pd.read_sql(qr, pconn)

,LANDING__OUTCOME,COUNT
0,Success (ground pad),3
1,Success (drone ship),5


### Reference Links

*   <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01&origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>

*   <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01&origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*   <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01&origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*   <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Hands-on Lab: Analyzing a real World Data Set</a>


## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


## Change log

| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
